<img src="assets/logos/Logo_endpoint.png" width=500></img>


## Code
code is in /containers/Shoot and /containers/Layout and each directory there containes the following files
    
- ### /containers/*/Makefile
Makefile for creating the tar.gz file of code for sagemaker framework containers

- ### /containers/*/host.py
The code for generating inferences

- ### /containers/*/test.py
code for testing the the host.py and train.py files localy

- ### /containers/*/train.py
the training code


## Deployment flow
<img src="assets/build/deployment-flow.png" width=800></img>



## Pipeline Config
Your pipelines are configured by two files, shoot-config.json and layout-config.json. You can edit things like hyperparamters, deployment environmental variables, and instance types. The following describes the properties you change/should change. 

|Parameter Name| Description | Suggested Value|
|---|---|---|
|frameworkversion|frame work version|"1.3.0"|
|inputmode|"File" or "Pipe" depending on what input mode you want|"File"|
|hyperparameters|key value hyperparameters to send to your training job|{}|
|channels|the data channels for the instance, the suggested value will create a training data channel pointing to data with the 'data/train' prefix in the data bucket|{"train":{"path":"data/train"}}|
|trainvolumesize|how much EBS storage to attach to the training instances|"5"|
|traininstancecount|how many instances you want to use for training|1|
|hostinstancetype|inference instance type|"ml.t2.medium"|
|traininstancetype|training instance type|"ml.m5.large"|

for for details on possible parameters and values see the [aws-sagemaker-build documentation](https://github.com/aws-samples/aws-sagemaker-build#Parameters). When you run the startPipeline.js script this config files will be merged into the SSM parameter store that configures the pipelines.

## Test

### Local
to test the training and inference of you model locally (before deploying) do:
- (optionaly) edit /containers/*/test.py to configure/customize your tests
- (If you model trains on data) download the data and move sample to mock directory
```shell
../../../bin/download_data.sh #downloads data from master Archive, parses it, and also create data sample
mkdir -p mock/data 
cp ../../../data/data.min.json mock/data/data.jsona #copy the data sample to the mock directory
```
- run test.py
```shell
test.py
```

if you get a message about missing dependencies (like mxnet) just install them with:
```shell
pip install mxnet # for example
```

### Deployed
to test the endpoint (your deployed model) run :
- (optionaly) edit /bin/testLayout.py or /bin/testShoot.py to configure/customize your test
- run 
```shell
/bin/testLayout.py # test for Defensive team
/bin/testShoot.py # test for Offensive team
```

## Pushing your changes
once you have made your changes and are ready to deploy them to production follow these steps
1. run the following command to stage files you change to be commited
```shell
git add <your file>
```
1. commit your changes with a message
```shell
git commit -m "<your message>"
```
1. push your changes to the remote repository
```shell
git push shoot #for offense
git push layout # for defense
```
1. inform your opperations that there are changes to deploy

## Data
All games are recorded and available in the game archive bucket. You will not be able to access the the data bucket until the opperations group has registered your team. 

### Download
The following script will download all the data from the S3 bucket (set in the config.js), unzip it, and merge it into a single file at /data/data.json
```shell
./bin/download_data.sh
```

### Format:
The /data/data.json file is a new line deliminated json file with the following format
```json
{                                                                                      
    "Status":"Finished",
    "winner":"B",
    "start_time":"<epoch time>",
    "end_time":"<epoch time>",
	"GameId":"d0d73e64-bd47-4bb2-ab50-ed0bd1935066",
    "TeamA":{
        "layout":[[0,0 ...],...],
        "score":0,
        "session":{},
        "name":"purple people eaters",
		"shots":[{"x":6,"y":1},....]
     },
     "TeamB":{
        "layout":[[0,0 ...],...],
        "score":0,
        "session":{},
        "name":"the dr strange loves",
		"shots":[{"x":1,"y":4},....]
	 }
}
```
|top level field|Description|
|---|---|
|Status| Finished or Failed depending on the status of the game|
|winner| the team name of the winner|
|start_time| when the game started|
|end_time| when the game finished|
|GameId| a unique ID for the game. Can be used with the replay web page to view a specific game|

|Team* Field|Description|
|---|---|
|layout|the layout of ships for this game|
|score| how many ship squares were hit by this team|
|session| the last session state for this game for this team|
|shots| an ordered list of shots for this game. shots[0] is the first shot, shots[1] is the second etc.|
|name| the team name for this team|

### Upload training data
after you have done any transformation to the data you will need to upload the data to the appropriate data bucket by running the following
```shell
./bin/download_data.json
```
```shell
./bin/upload_data.json
```

## Push and Commit
When you have a finish write new code to train and eploy your model commit the changes to your git branch and tell the opperations team to merge in your changes and deploy your code. 

```shell
git commit -a -m "I am adding these features <these features>"
git push -u origin $(git branch-name)
```